In [ ]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.8/116.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.2/661.2 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.0 MB/s eta 0:00:00


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from agents import (
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
)
from google.colab import userdata

gemini_api_key = userdata.get("GEMINI_API_KEY")


# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

from agents import set_default_openai_client, set_tracing_disabled

set_default_openai_client(external_client)
set_tracing_disabled(True)

In [ ]:
import asyncio
from agents import Agent, Runner, ItemHelpers, MessageOutputItem, trace

spanish_agent = Agent(
    name="spanish_agent",
    instructions="You translate the user's message to Spanish",
    handoff_description="An english to spanish translator",
    model=model
)

french_agent = Agent(
    name="french_agent",
    instructions="You translate the user's message to French",
    handoff_description="An english to french translator",
    model=model
)

italian_agent = Agent(
    name="italian_agent",
    instructions="You translate the user's message to Italian",
    handoff_description="An english to italian translator",
    model=model
)

orchestrator_agent = Agent(
    name="orchestrator_agent",
    instructions=(
        "You are a translation agent. You use the tools given to you to translate."
        "If asked for multiple translations, you call the relevant tools in order."
        "You never translate on your own, you always use the provided tools."
    ),
    tools=[
        spanish_agent.as_tool(
            tool_name="translate_to_spanish",
            tool_description="Translate the user's message to Spanish",
        ),
        french_agent.as_tool(
            tool_name="translate_to_french",
            tool_description="Translate the user's message to French",
        ),
        italian_agent.as_tool(
            tool_name="translate_to_italian",
            tool_description="Translate the user's message to Italian",
        ),
    ],
    model=model
)

async def main():
    msg = input("Hi! What would you like translated, and to which languages? ")
    result = await Runner.run(orchestrator_agent, msg)
    print(f"\n\nFinal response:\n{result.final_output}")

if __name__ == "__main__":
    asyncio.run(main())

Hi! What would you like translated, and to which languages? In spanish, Hello


Final response:
Hola.



In [ ]:
from agents import (
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
    Agent,
    Runner
)
from google.colab import userdata
import asyncio

class TranslationAgent:
    def __init__(self, api_key, model_name="gemini-2.0-flash"):
        self.api_key = api_key
        self.model_name = model_name
        self.external_client = self._create_external_client()
        self.model = self._create_model()

    def _create_external_client(self):
        return AsyncOpenAI(
            api_key=self.api_key,
            base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
        )

    def _create_model(self):
        return OpenAIChatCompletionsModel(
            model=self.model_name,
            openai_client=self.external_client
        )

    def create_agent(self, name, instructions, handoff_description):
        return Agent(
            name=name,
            instructions=instructions,
            handoff_description=handoff_description,
            model=self.model
        )

    def create_orchestrator_agent(self, tools):
        instructions = (
            "You are a translation agent. You use the tools given to you to translate."
            "If asked for multiple translations, you call the relevant tools in order."
            "You never translate on your own, you always use the provided tools."
        )
        return Agent(
            name="orchestrator_agent",
            instructions=instructions,
            tools=tools,
            model=self.model
        )

    async def run_translation(self, orchestrator_agent, message):
        result = await Runner.run(orchestrator_agent, message)
        return result.final_output

# Get API key
gemini_api_key = userdata.get("GEMINI_API_KEY")
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

# Create translation agent instance
translator = TranslationAgent(gemini_api_key)

# Create individual translator agents
spanish_agent = translator.create_agent(
    name="spanish_agent",
    instructions="You translate the user's message to Spanish",
    handoff_description="An english to spanish translator"
)
french_agent = translator.create_agent(
    name="french_agent",
    instructions="You translate the user's message to French",
    handoff_description="An english to french translator"
)
italian_agent = translator.create_agent(
    name="italian_agent",
    instructions="You translate the user's message to Italian",
    handoff_description="An english to italian translator"
)

# Create orchestrator agent
orchestrator_agent = translator.create_orchestrator_agent(tools=[
    spanish_agent.as_tool(tool_name="translate_to_spanish", tool_description="Translate the user's message to Spanish"),
    french_agent.as_tool(tool_name="translate_to_french", tool_description="Translate the user's message to French"),
    italian_agent.as_tool(tool_name="translate_to_italian", tool_description="Translate the user's message to Italian"),
])

# Run translation
async def main():
    msg = input("Hi! What would you like translated, and to which languages? ")
    translated_text = await translator.run_translation(orchestrator_agent, msg)
    print(f"\n\nFinal response:\n{translated_text}")

if __name__ == "__main__":
    asyncio.run(main())

Hi! What would you like translated, and to which languages? In Spanish, Hello


Final response:
Hola.

